In [2]:
def convert_txt_to_json(txt_path, json_path, names):
    import json
    data = []
    with open(txt_path, 'r', encoding='utf-8') as f:
        for idx, text in enumerate(f.read().split('\n\n')):
            ner_tags = []
            sample = {}
            tokens = []
            for line in text.split('\n'):
                if not line.strip():
                    continue
                token_tag = line.split()
                if len(token_tag) != 2:
                    continue
                token, tag = token_tag
                tokens.append(token)
                if tag not in names:
                    names.append(tag)
                ner_tags.append(names.index(tag))
            sample['id'] = idx
            sample['tokens'] = tokens
            sample['ner_tags'] = ner_tags
            data.append(sample)
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    return names

In [3]:
# 用法示例
names = []
names = convert_txt_to_json('../data/medical/train.txt', '../data/medical/trian.json', names)
print(names)
names = convert_txt_to_json('../data/medical/dev.txt', '../data/medical/dev.json', names)
print(names)
names = convert_txt_to_json('../data/medical/test.txt', '../data/medical/test.json', names)
print(names)

['O', 'B-临床表现', 'I-临床表现', 'B-中医治疗', 'I-中医治疗', 'B-西医诊断', 'I-西医诊断', 'B-方剂', 'I-方剂', 'B-中药', 'I-中药', 'B-中医诊断', 'I-中医诊断', 'B-西医治疗', 'I-西医治疗', 'B-中医证候', 'I-中医证候', 'B-中医治则', 'I-中医治则', 'B-其他治疗', 'I-其他治疗']
['O', 'B-临床表现', 'I-临床表现', 'B-中医治疗', 'I-中医治疗', 'B-西医诊断', 'I-西医诊断', 'B-方剂', 'I-方剂', 'B-中药', 'I-中药', 'B-中医诊断', 'I-中医诊断', 'B-西医治疗', 'I-西医治疗', 'B-中医证候', 'I-中医证候', 'B-中医治则', 'I-中医治则', 'B-其他治疗', 'I-其他治疗']
['O', 'B-临床表现', 'I-临床表现', 'B-中医治疗', 'I-中医治疗', 'B-西医诊断', 'I-西医诊断', 'B-方剂', 'I-方剂', 'B-中药', 'I-中药', 'B-中医诊断', 'I-中医诊断', 'B-西医治疗', 'I-西医治疗', 'B-中医证候', 'I-中医证候', 'B-中医治则', 'I-中医治则', 'B-其他治疗', 'I-其他治疗']


In [5]:
from datasets import load_dataset, Value, Features, ClassLabel, Sequence

data_files = {'train': '../data/medical/train.json', 'dev': '../data/medical/dev.json', 'test': '../data/medical/test.json'}
features = Features({
    'id': Value('int32'),
    'tokens': Sequence(Value('string')),
    'ner_tags': Sequence(ClassLabel(num_classes=21, names=names))
})

raw_dataset = load_dataset('json', data_files=data_files, features=features)

Generating train split: 5259 examples [00:00, 91450.06 examples/s]
Generating dev split: 657 examples [00:00, 94944.11 examples/s]
Generating test split: 658 examples [00:00, 91631.60 examples/s]


In [6]:
raw_dataset['dev'].features

{'id': Value(dtype='int32', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-临床表现', 'I-临床表现', 'B-中医治疗', 'I-中医治疗', 'B-西医诊断', 'I-西医诊断', 'B-方剂', 'I-方剂', 'B-中药', 'I-中药', 'B-中医诊断', 'I-中医诊断', 'B-西医治疗', 'I-西医治疗', 'B-中医证候', 'I-中医证候', 'B-中医治则', 'I-中医治则', 'B-其他治疗', 'I-其他治疗'], id=None), length=-1, id=None)}

In [8]:
raw_dataset['dev']['ner_tags'][0]

[0,
 7,
 8,
 8,
 8,
 8,
 0,
 0,
 0,
 9,
 10,
 0,
 9,
 10,
 0,
 0,
 0,
 0,
 0,
 9,
 10,
 10,
 0,
 9,
 10,
 10,
 0,
 0,
 0,
 0,
 9,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 10,
 0,
 9,
 10,
 0,
 0,
 0,
 0,
 0,
 9,
 10,
 10,
 0,
 0,
 0,
 0,
 9,
 10,
 0,
 0]

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
HF_ENDPOINT = os.getenv('HF_ENDPOINT')
HF_ENDPOINT

'https://hf-mirror.com'

In [1]:
!huggingface-cli env


Copy-and-paste the text below in your GitHub issue.

- huggingface_hub version: 0.33.2
- Platform: macOS-15.1.1-arm64-arm-64bit
- Python version: 3.12.11
- Running in iPython ?: No
- Running in notebook ?: No
- Running in Google Colab ?: No
- Running in Google Colab Enterprise ?: No
- Token path ?: /Users/zhouke/.cache/huggingface/token
- Has saved token ?: False
- Configured git credential helpers: osxkeychain
- FastAI: N/A
- Tensorflow: N/A
- Torch: 2.7.1
- Jinja2: 3.1.6
- Graphviz: N/A
- keras: N/A
- Pydot: N/A
- Pillow: 11.3.0
- hf_transfer: N/A
- gradio: N/A
- tensorboard: N/A
- numpy: 2.3.1
- pydantic: N/A
- aiohttp: 3.12.13
- hf_xet: 1.1.5
- ENDPOINT: https://hf-mirror.com
- HF_HUB_CACHE: /Users/zhouke/.cache/huggingface/hub
- HF_ASSETS_CACHE: /Users/zhouke/.cache/huggingface/assets
- HF_TOKEN_PATH: /Users/zhouke/.cache/huggingface/token
- HF_STORED_TOKENS_PATH: /Users/zhouke/.cache/huggingface/stored_tokens
- HF_HUB_OFFLINE: False
- HF_HUB_DISABLE_TELEMETRY: False
- HF_HUB_DIS

In [2]:
from transformers import AutoTokenizer, AutoModel

checkpoint = 'Qwen/Qwen2.5-7B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)

/Users/zhouke/Documents/project/stage3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

{"timestamp":"2025-07-03T08:20:58.192069Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, source: hyper_util::client::legacy::Error(SendRequest, hyper::Error(IncompleteMessage)) }). Retrying..."},"filename":"/Users/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":242}
{"timestamp":"2025-07-03T08:20:58.192095Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 2.416646533s before the next attempt"},"filename":"/Users/runner/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}


Fetching 4 files:   0%|          | 0/4 [01:17<?, ?it/s]
Cancellation requested; stopping current tasks.


KeyboardInterrupt: 